In [1]:
# required libraries
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import pickle

# create a function to scrape team performance for multiple years

to get rosters

start with the year -

https://www.basketball-reference.com/leagues/NBA_YEAR_ratings

loop through each team

https://www.basketball-reference.com/teams/THIS_TEAM/2014.html

get players from roster -
note- no per per game so lets use per from previous season

player column of roster
which contains a link to the players profile
https://www.basketball-reference.com/players/p/pendeje02.html

<a href="/players/p/pendeje02.html">Jeff Ayres</a>





In [73]:
def team_roster_base(years = [2015]):
    team_roster_base = pd.DataFrame(columns = ["Year", "Team", "team_dir"])
    # loop through each year
    for y in years:
        # NBA season to scrape- year is season end so 2015 is 2014-15 season 
        year = y
        url = f"https://www.basketball-reference.com/leagues/NBA_{year}_ratings.html"
        html = urlopen(url)
        soup = BeautifulSoup(html, features="lxml")
        table = soup.find('table', attrs={'id':'ratings'})
        teams = table.tbody.findAll("tr")
        for team in teams: #get team names and links - would get stats here as well
            team_name= team.td.string
            team_dir = team.td.a.get('href')
            team_year={"Year": year, "Team": team_name, "team_dir": team_dir}
            team_roster_base = team_roster_base.append(team_year, ignore_index = True)
        time.sleep(5)
    # export to csv
    team_roster_base.to_csv("year_team_link.csv", index=False)
    return team_roster_base

def get_rosters(team_roster_base):
    save_int = 60
    save = 0
    yearly_rosters = pd.DataFrame(columns = ["Year", "Team", "team_dir", "Player", "player_dir"])
    base_url = "https://www.basketball-reference.com"
    for index, row in team_roster_base.iterrows():
        roster_url = base_url+str(row['team_dir'])
        time.sleep(random.randint(3, 9))
        html_team = urlopen(roster_url)
        soup = BeautifulSoup(html_team, features="lxml")
        roster_table = soup.find('table', attrs={'id':'roster'})
        players = roster_table.tbody.findAll("tr")
        year = [str(row['Year'])]
        team = [str(row['Team'])]
        team_dir = [str(row['team_dir'])]
        for player in players:
            player_name = [str(player.td.string)]
            player_dir = [str(player.td.a.get('href'))]
            team_year_player = pd.DataFrame({"Year": year,
                                             "Team": team, 
                                             "team_dir": team_dir, 
                                             "Player": player_name, 
                                             "player_dir": player_dir})
            yearly_rosters = pd.concat([yearly_rosters, team_year_player], ignore_index = True)
            save+=1
        if save >= save_int:
            save = 0
            print(len(yearly_rosters))
            with open("df.pickle", "wb") as file:
                pickle.dump(yearly_rosters, file)
    with open("df.pickle", "wb") as file:
                pickle.dump(yearly_rosters, file)
    yearly_rosters.to_csv("season_rosters.csv", index=False)
    return yearly_rosters

# seasons = team_roster_base(years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])
seasons = pd.read_csv("year_team_link.csv")
team_rosters = get_rosters(seasons)

KeyboardInterrupt: 

In [64]:
def get_pers(season_rosters, filename):
    #get player efficiency ratings 
    progress = 0 
    save = 0
    per_table = pd.DataFrame(columns = ["season", "team_id", "player_dir", "per"])
    save_int = 50   
    players_list = []
    for index, row in season_rosters.iterrows():
        progress+= 1
        season = str(int(row['Year'])-1)+'-'+row['Year'][2:]
        team = row['team_dir'][7:10]
        base_url = "https://www.basketball-reference.com"
        player_url = base_url+str(row['player_dir'])
        if row['player_dir'] in players_list:
            continue
        else:
            players_list.append(row['player_dir'])
        try:
            time.sleep(random.randint(3, 9))
            html_player = urlopen(player_url)
            player_seasons = []
            player_team_ids = []
            # player_team_dirs = []
            player_dirs = []
            player_pers = []
            soup = BeautifulSoup(html_player, features="lxml")
            adv_table = soup.find('table', attrs={'id':'advanced'})
            teams = adv_table.tbody.findAll("tr")
            for team in teams:
                # th is season td is all other stats 
                player_seasons.append(str(team.th.a.string))
                print(player_seasons)
                other_stats = team.findAll('td')
                team_id = other_stats[1].string
                player_team_ids.append(str(team_id))
                # player_team_dirs.append(row['team_dir'])
                player_dirs.append(row['player_dir'])
                per_value = other_stats[6].string
                player_pers.append(per_value)
            this_player = pd .DataFrame({ "season": player_seasons,
                                         "team_id": player_team_ids, 
                                         "player_dir": player_dirs, 
                                         "per":player_pers})
            per_table = pd.concat([per_table, this_player], ignore_index = True)
            save+=1
        except:
            print(row)
        if save >= save_int:
            save = 0
            print(len(per_table), end = " ")
#changing filenames for second run 
            per_table.to_parquet(f'{filename}.parquet.gzip',
              compression='gzip')
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_parquet.html
    per_table.to_parquet(f'{filename}.parquet.gzip',
              compression='gzip')
        
    # per_table.to_csv("PER_table.csv", index=False)
    return per_table

In [74]:
# players on our rosters
season_rosters = pd.read_pickle("df.pickle", compression='infer')
# pers = get_pers(season_rosters)
season_rosters_players = season_rosters.drop_duplicates(subset = ["player_dir"], keep = "first")
players_all = season_rosters_players["player_dir"]
len(players_all)

1240

In [78]:
#player pers every season up to the error
season_PER = pd.read_parquet('PER_table.parquet.gzip') 
season_PER= season_PER.drop_duplicates(subset = ["player_dir"], keep = "first")
players_have = season_PER["player_dir"]
players_have

0       /players/b/barbole01.html
16      /players/b/barneha02.html
30      /players/b/bogutan01.html
46      /players/c/curryst01.html
61      /players/e/ezelife01.html
                  ...            
8864    /players/b/brookar01.html
8869     /players/l/lamban01.html
8872    /players/m/martike04.html
8878    /players/o/oliveca01.html
8880     /players/t/tateja01.html
Name: player_dir, Length: 1100, dtype: object

In [79]:
#players missing due to the error
players_needed = set(players_all).difference(set(players_have))
len(set(players_all).difference(set(players_have)))

140

In [80]:
#selecting the players we dont have 
missing_players = season_rosters_players[season_rosters_players["player_dir"].isin(players_needed)]
len(missing_players)
missing_players

,Year,Team,team_dir,Player,player_dir
4012,2021,Houston Rockets,/teams/HOU/2021.html,Brodric Thomas,/players/t/thomabr01.html
4034,2021,Cleveland Cavaliers,/teams/CLE/2021.html,Isaac Okoro,/players/o/okorois01.html
4038,2021,Cleveland Cavaliers,/teams/CLE/2021.html,Lamar Stevens,/players/s/stevela01.html
4042,2021,Cleveland Cavaliers,/teams/CLE/2021.html,Dylan Windler,/players/w/windldy01.html
4044,2021,Orlando Magic,/teams/ORL/2021.html,Cole Anthony,/players/a/anthoco01.html
...,...,...,...,...,...
4783,2022,Portland Trail Blazers,/teams/POR/2022.html,Greg Brown III,/players/b/browngr01.html
4785,2022,Portland Trail Blazers,/teams/POR/2022.html,Jarron Cumberland,/players/c/cumbeja01.html
4796,2022,Portland Trail Blazers,/teams/POR/2022.html,Cameron McGriff,/players/m/mcgrica01.html
4805,2022,Portland Trail Blazers,/teams/POR/2022.html,Trendon Watford,/players/w/watfotr01.html


In [81]:
# scraping the missing players into missing pers
missing_players_df = get_pers(missing_players, "missing_players")

Year                               2022
Team                    Milwaukee Bucks
team_dir           /teams/MIL/2022.html
Player                     Luca Vildoza
player_dir    /players/v/vildolu01.html
Name: 4285, dtype: object
151 276 

In [82]:
# players after scraping attempt for missing 
mp = pd.read_parquet('missing_players.parquet.gzip') 
mp= mp.drop_duplicates(subset = ["player_dir"], keep = "first")
players_mp = mp["player_dir"]
players_mp
#merge with existing
players_have = pd.concat([players_have, players_mp], ignore_index = True)

In [83]:
#players still missing
players_needed = set(players_all).difference(set(players_have))
len(set(players_all).difference(set(players_have)))

1

In [84]:
#selecting the player we dont have 
missing_players = season_rosters_players[season_rosters_players["player_dir"].isin(players_needed)]
len(missing_players)
missing_players

,Year,Team,team_dir,Player,player_dir
4285,2022,Milwaukee Bucks,/teams/MIL/2022.html,Luca Vildoza,/players/v/vildolu01.html


In [86]:
luca_vildoza = pd.DataFrame({ "season": [2021-22],
                                         "team_id": ["MIL"], 
                                         "player_dir": ["/players/v/vildolu01.html"], 
                                         "per":[17.9]})
season_player_per = pd.concat([pd.read_parquet('PER_table.parquet.gzip'), pd.read_parquet('missing_players.parquet.gzip') , luca_vildoza], ignore_index = True)
# season_player_per.drop_duplicates(subset = ["player_dir"], keep = "first")
# season_player_per.to_csv("seasons_per_s.csv", index=False)
#need to remove season pers before the year 2015
season_player_per = season_player_per[season_player_per["season"].isin(["2014-15", "2015-16", "2016-17", "2017-18", "2018-19", "2019-20", "2020-21", "2021-22"])]
#this list is larger because of trades/transfers etc
season_player_per.to_csv("seasons_per_s.csv", index=False)
season_player_per

,season,team_id,player_dir,per
13,2014-15,GSW,/players/b/barbole01.html,15.3
14,2015-16,GSW,/players/b/barbole01.html,11.7
15,2016-17,PHO,/players/b/barbole01.html,11.5
18,2014-15,GSW,/players/b/barneha02.html,13.4
19,2015-16,GSW,/players/b/barneha02.html,12.3
...,...,...,...,...
9245,2021-22,POR,/players/b/browngr01.html,11.0
9248,2021-22,POR,/players/c/cumbeja01.html,8.4
9249,2021-22,POR,/players/m/mcgrica01.html,12.5
9250,2021-22,POR,/players/w/watfotr01.html,15.8


In [70]:
season_rosters = pd.read_pickle("df.pickle", compression='infer')
season_rosters 

,Year,Team,team_dir,Player,player_dir
0,2015,Golden State Warriors,/teams/GSW/2015.html,Leandro Barbosa,/players/b/barbole01.html
1,2015,Golden State Warriors,/teams/GSW/2015.html,Harrison Barnes,/players/b/barneha02.html
2,2015,Golden State Warriors,/teams/GSW/2015.html,Andrew Bogut,/players/b/bogutan01.html
3,2015,Golden State Warriors,/teams/GSW/2015.html,Stephen Curry,/players/c/curryst01.html
4,2015,Golden State Warriors,/teams/GSW/2015.html,Festus Ezeli,/players/e/ezelife01.html
...,...,...,...,...,...
4804,2022,Portland Trail Blazers,/teams/POR/2022.html,Tony Snell,/players/s/snellto01.html
4805,2022,Portland Trail Blazers,/teams/POR/2022.html,Trendon Watford,/players/w/watfotr01.html
4806,2022,Portland Trail Blazers,/teams/POR/2022.html,Brandon Williams,/players/w/willibr03.html
4807,2022,Portland Trail Blazers,/teams/POR/2022.html,Justise Winslow,/players/w/winslju01.html


# next step get team stats- how are we defining team stats 
#we need team stats for each game player over the regular season by each team. 

#1 scrape season matchups from https://www.basketball-reference.com/leagues/NBA_2015_games-MONTH.html (OCTOBER- APRIL(PLAYOFFS))
months = ["october", "november", "december", "january", "february", "march", "april"]
f"https://www.basketball-reference.com/leagues/NBA_2015_games-{month}.html"
for month in month:
schedule_table = soup.find('table', attrs={'id':'schedule'})
this_game = adv_table.tbody.findAll("tr")
####### make sure its before the playoffs row in april 
save team abreviations from link "/team/TEAM ID/year.html, score? home or away 
<!-- ####### save arena, team as home or away score  -->

#2 USE BOX SCORE LINK TO GET INDIVIDUAL GAME STAS 
EX. https://www.basketball-reference.com/boxscores/201504010CHO.html 201504010CHO- DIFFERENT FOR EACH GAME 

####### use team_id's to identify team stats table from the game this table also has player stats if we decde to use them 
ex. soup.find('table', attrs={'id':"box-TEAM_ID-game-basic"})
get team stats from the tfoot tag

#3 save data - probably should do this in seasonal increments 

In [161]:
def get_season_schedule(filename, seasons = [2015]):
    season_schedule_table = pd.DataFrame(columns = ["date", "away_team_id", "home_team_id", "arena", "link"])
    months = ["october", "november", "december", "january", "february", "march", "april"]
    for year in seasons:
        for month in months:
            month_schedule = f"https://www.basketball-reference.com/leagues/NBA_{year}_games-{month}.html"
            print(month_schedule)
            time.sleep(random.randint(2, 6))
            month_schedule_link = urlopen(month_schedule)
            soup = BeautifulSoup(month_schedule_link, features="lxml")
            schedule_table = soup.find('table', attrs={'id':'schedule'})
            games = schedule_table.tbody.findAll("tr")
            dates = []
            away_team_ids = []
            home_team_ids = []
            arenas = []
            links = []
            for game in games:
            #avoinding playoffs schedule
                if game.th.string == "Playoffs":
                    season_month = pd.DataFrame({"date": dates,
                                         "away_team_id": away_team_ids, 
                                         "home_team_id": home_team_ids, 
                                          "arena":arenas,
                                         "link":links
                                        })
                    break
                    
                other_stats = game.findAll("td")
                date = game.th.a.string
                dates.append(date)
                away_team = other_stats[1].a.get('href')[7:10]
                away_team_ids.append(away_team)
                home_team = other_stats[3].a.get('href')[7:10]
                home_team_ids.append(home_team)
                boxscore = other_stats[5].a.get('href')
                links.append(boxscore)
                arena = other_stats[8].string
                arenas.append(arena)
            season_month = pd.DataFrame({"date": dates,
                                         "away_team_id": away_team_ids, 
                                         "home_team_id": home_team_ids, 
                                          "arena":arenas,
                                         "link":links
                                        })
            try:
                name_of_file = f"{filename}.parquet.gzip"
                season_games = pd.concat([pd.read_parquet(name_of_file), season_month], ignore_index = True)
                season_games.to_parquet(name_of_file)              
            except:
                name_of_file = f"{filename}.parquet.gzip"
                season_month.to_parquet(name_of_file)
            print(month, "completed")

In [162]:
# 2015 to start 
# years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
seasons = [2015]
get_season_schedule(str(seasons), seasons)

https://www.basketball-reference.com/leagues/NBA_2015_games-october.html
october completed
https://www.basketball-reference.com/leagues/NBA_2015_games-november.html
november completed
https://www.basketball-reference.com/leagues/NBA_2015_games-december.html
december completed
https://www.basketball-reference.com/leagues/NBA_2015_games-january.html
january completed
https://www.basketball-reference.com/leagues/NBA_2015_games-february.html
february completed
https://www.basketball-reference.com/leagues/NBA_2015_games-march.html
march completed
https://www.basketball-reference.com/leagues/NBA_2015_games-april.html
april completed


In [222]:
name_of_file = f"{str(seasons)}.parquet.gzip"
season_games = pd.read_parquet(name_of_file)
season_games

,date,away_team_id,home_team_id,arena,link
0,"Tue, Oct 28, 2014",ORL,NOP,Smoothie King Center,/boxscores/201410280NOP.html
1,"Tue, Oct 28, 2014",DAL,SAS,AT&T Center,/boxscores/201410280SAS.html
2,"Tue, Oct 28, 2014",HOU,LAL,STAPLES Center,/boxscores/201410280LAL.html
3,"Wed, Oct 29, 2014",MIL,CHO,Time Warner Cable Arena,/boxscores/201410290CHO.html
4,"Wed, Oct 29, 2014",PHI,IND,Bankers Life Fieldhouse,/boxscores/201410290IND.html
...,...,...,...,...,...
2575,"Wed, Apr 15, 2015",DET,NYK,Madison Square Garden (IV),/boxscores/201504150NYK.html
2576,"Wed, Apr 15, 2015",MIA,PHI,Wells Fargo Center,/boxscores/201504150PHI.html
2577,"Wed, Apr 15, 2015",IND,MEM,FedEx Forum,/boxscores/201504150MEM.html
2578,"Wed, Apr 15, 2015",DEN,GSW,Oracle Arena,/boxscores/201504150GSW.html


In [230]:
def get_game_stats(dataframe, filename):
    count = 0 
    for game in dataframe.iterrows():
        game = game[1]
        home_team_id = game["home_team_id"]
        away_team_id = game["away_team_id"]
        arena = game["arena"]
        ext = game["link"]
        box_score_url = f"https://www.basketball-reference.com{ext}"
        time.sleep(random.randint(2, 6))
        open_link = urlopen(box_score_url)
        soup = BeautifulSoup(open_link, features="lxml")
        home_stats = soup.find('table', attrs={'id':f"box-{home_team_id}-game-basic"}).tfoot.tr
        home_fg = home_stats.find('td', attrs = {'data-stat': 'fg'}).string
        home_fga = home_stats.find('td', attrs = {'data-stat': 'fga'}).string
        home_fg_pct = home_stats.find('td', attrs = {'data-stat': 'fg_pct'}).string
        home_fg3 = home_stats.find('td', attrs = {'data-stat': 'fg3'}).string
        home_fg3a = home_stats.find('td', attrs = {'data-stat': 'fg3a'}).string
        home_fg3_pct = home_stats.find('td', attrs = {'data-stat': 'fg3_pct'}).string
        home_ft = home_stats.find('td', attrs = {'data-stat': 'ft'}).string
        home_fta = home_stats.find('td', attrs = {'data-stat': 'fta'}).string
        home_ft_pct = home_stats.find('td', attrs = {'data-stat': 'ft_pct'}).string
        home_orb = home_stats.find('td', attrs = {'data-stat': 'orb'}).string
        home_drb = home_stats.find('td', attrs = {'data-stat': 'drb'}).string
        home_trb = home_stats.find('td', attrs = {'data-stat': 'trb'}).string
        home_ast = home_stats.find('td', attrs = {'data-stat': 'ast'}).string
        home_stl = home_stats.find('td', attrs = {'data-stat': 'stl'}).string
        home_blk = home_stats.find('td', attrs = {'data-stat': 'blk'}).string
        home_tov = home_stats.find('td', attrs = {'data-stat': 'tov'}).string
        home_pf = home_stats.find('td', attrs = {'data-stat': 'pf'}).string
        home_pts = home_stats.find('td', attrs = {'data-stat': 'pts'}).string
        
        away_stats = soup.find('table', attrs={'id':f"box-{away_team_id}-game-basic"}).tfoot.tr
        away_fg = away_stats.find('td', attrs = {'data-stat': 'fg'}).string
        away_fga = away_stats.find('td', attrs = {'data-stat': 'fga'}).string
        away_fg_pct = away_stats.find('td', attrs = {'data-stat': 'fg_pct'}).string
        away_fg3 = away_stats.find('td', attrs = {'data-stat': 'fg3'}).string
        away_fg3a = away_stats.find('td', attrs = {'data-stat': 'fg3a'}).string
        away_fg3_pct = away_stats.find('td', attrs = {'data-stat': 'fg3_pct'}).string
        away_ft = away_stats.find('td', attrs = {'data-stat': 'ft'}).string
        away_fta = away_stats.find('td', attrs = {'data-stat': 'fta'}).string
        away_ft_pct = away_stats.find('td', attrs = {'data-stat': 'ft_pct'}).string
        away_orb = away_stats.find('td', attrs = {'data-stat': 'orb'}).string
        away_drb = away_stats.find('td', attrs = {'data-stat': 'drb'}).string
        away_trb = away_stats.find('td', attrs = {'data-stat': 'trb'}).string
        away_ast = away_stats.find('td', attrs = {'data-stat': 'ast'}).string
        away_stl = away_stats.find('td', attrs = {'data-stat': 'stl'}).string
        away_blk = away_stats.find('td', attrs = {'data-stat': 'blk'}).string
        away_tov = away_stats.find('td', attrs = {'data-stat': 'tov'}).string
        away_pf = away_stats.find('td', attrs = {'data-stat': 'pf'}).string
        away_pts = away_stats.find('td', attrs = {'data-stat': 'pts'}).string
    
        game_stats = pd.DataFrame({
        "home_team":[home_team_id],
        "home_fg":[home_fg], 
        "home_fga":[home_fga], 
        "home_fg_pct":[home_fg_pct], 
        "home_fg3":[home_fg3], 
        "home_fg3a":[home_fg3a], 
        "home_fg3_pct":[home_fg3_pct], 
        "home_ft":[home_ft], 
        "home_fta":[home_fta], 
        "home_ft_pct":[home_ft_pct], 
        "home_orb":[home_orb], 
        "home_drb":[home_drb],
        "home_trb":[home_trb],
        "home_ast":[home_ast],
        "home_stl":[home_stl],
        "home_blk":[home_blk],
        "home_tov":[home_tov],
        "home_pf":[home_pf],
        "home_pts":[home_pts],
        "away_team":[away_team_id],
        "away_fg":[away_fg], 
        "away_fga":[away_fga], 
        "away_fg_pct":[away_fg_pct], 
        "away_fg3":[away_fg3], 
        "away_fg3a":[away_fg3a], 
        "away_fg3_pct":[away_fg3_pct], 
        "away_ft":[away_ft], 
        "away_fta":[away_fta], 
        "away_ft_pct":[away_ft_pct], 
        "away_orb":[away_orb], 
        "away_drb":[away_drb],
        "away_trb":[away_trb],
        "away_ast":[away_ast],
        "away_stl":[away_stl],
        "away_blk":[away_blk],
        "away_tov":[away_tov],
        "away_pf":[away_pf],
        "away_pts":[away_pts],
        "arena": [arena]})
        if count % 100 == 0:
            print(count)   
            try:
                name_of_file = f"{filename}.parquet.gzip"
                game_stats = pd.concat([pd.read_parquet(name_of_file), game_stats], ignore_index = True)
                game_stats.to_parquet(name_of_file)              
            except:
                name_of_file = f"{filename}.parquet.gzip"
                game_stats.to_parquet(name_of_file)
            count+=1
    name_of_file = f"{filename}.parquet.gzip"
    game_stats = pd.concat([pd.read_parquet(name_of_file), game_stats], ignore_index = True)
    game_stats.to_parquet(name_of_file) 

In [ ]:
get_game_stats(season_games, "2014-15_game_stats")

0
